
## Check GMF integrity


In [1]:
import os
import cv2
import numpy as np
import time


In [ ]:

start = time.time()
assert os.system("python inference.py") == 0
end = time.time()
print("Time elapsed: ", end - start)

In [21]:
# read check.png and ref.png and compare them with np
out = cv2.imread("check.png")
ref = cv2.imread("ref.png")
np.allclose(out, ref, atol=1, equal_nan=True)

True

In [23]:
gmf_ncnn_out = cv2.imread(r"D:\Program\VSsource\research\gmfss-ncnn-vulkan-demo\images\out.png")

## gen pt

In [32]:
if os.path.exists("flownet_288.pt"):
    os.remove("flownet_288.pt")
assert os.system("python gen_flow_pt.py") == 0
os.system("pnnx flownet_288.pt")

KeyboardInterrupt: 

In [24]:
import numpy as np
output = np.load("output.npy")
ref = np.load("ref_output.npy")
np.allclose(output, ref, atol=1, equal_nan=True)

# import torchvision.models as models
# models.swin_t()

False

.\pnnx.exe D:/60-fps-Project/VFI/GMFSS2NCNN/flownet_288.pt inputshape=[1,3,288,480]f32,[1,3,288,480]f32 device=cpu moduleop=model.gmflow.utils.split_feature,model.gmflow.utils.merge_splits,model.gmflow.utils.convex_upsampling

## Clean pnnx.py

In [ ]:
with open("flownet_288_pnnx.py", 'r', encoding='utf-8') as r:
    content = r.read()
content = content.replace("", "")
with open("flownet_288_pnnx.py", 'w', encoding='utf-8') as w:
    w.write(content)


# Edit NCNN Param

nn\.InstanceNorm2d(\s+)(.*?)(\s+)([\d+\s]+)
InstanceNorm$1$2$3$4 2=0

after:
```python
v_964 = F.upsample(input=v_963, align_corners=True, mode='bilinear', scale_factor=(2.000000,2.000000))
v_965 = (v_964 * 2)
```
all split, merge, s=8

In [22]:
import os, shutil, re
param_path = "flownet_288.ncnn.param"
param_bak_path = "flownet_288.ncnn_bak.param"
if os.path.exists(param_bak_path):
    os.remove(param_bak_path)
shutil.copy(param_path, param_bak_path)
content = ""
is_s_8 = False
with open(param_bak_path, 'r', encoding='utf-8') as r:
    for l in r.readlines():
        if "Interp" in l:
            is_s_8 = True
        if re.search("nn.InstanceNorm2d", l, re.I):
                l = l.replace("nn.InstanceNorm2d", "InstanceNorm")
                l = f"{l.strip()} 2=0\n"
        if is_s_8:
            if re.search("split_feature|merge_splits", l, re.I):
                l = f"{l.strip()} 0=8\n"
        if re.search("convex_upsampling", l, re.I):
            l = f"{l.strip()} 0=4\n"
        content += l

with open(param_path, 'w', encoding='utf-8') as w:
    w.write(content)

In [7]:
print(len(content))

140530


# Compare for Mark

In [30]:
import re
import numpy as np
torch_output_mat = "debug0_in0_padded.txt"
ncnn_output_mat = "out_ncnn.txt"

class PrintData:
    def __init__(self, r) -> None:
        shape_info = r.readline()  # shape 960 540 1 2
        shape_info = re.findall(r"shape (\d+) (\d+) (\d+) (\d+)", shape_info)[0]
        self.w, self.h, self.d, self.c = map(int, shape_info)
        data = r.readline().split(' ')[:-1]
        self.data = np.array(data).reshape(self.c, self.d, self.h, self.w).astype(np.float32)

with open(torch_output_mat, 'r', encoding='utf-8') as r:
    torch_output = PrintData(r)
# with open(ncnn_output_mat, 'r', encoding='utf-8') as r:
#     ncnn_output = PrintData(r)


In [31]:
torch_output.data[0, 0, 0, 0:5]
# torch_output.data.shape

array([1.427, 1.427, 1.448, 1.449, 1.41 ], dtype=float32)

# Reference

In [6]:
import torch
import torchvision.models as models

net = models.resnet18(pretrained=False)
net = net.eval()

x = torch.rand(1, 3, 224, 224)

# You could try disabling checking when tracing raises error
# mod = torch.jit.trace(net, x, check_trace=False)
mod = torch.jit.trace(net, x)

mod.save("resnet18.pt")

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
